# Test Notebook

This is mainly a test notebook that runs db operations on the small test db (DEV connection) that is first created, then filled with 3 records and 2 'addition' credit trx, before a record is deactivated and then also fully deleted. Assert statements help to check everything works as expected.

But you can also use it to test front end manipulations if you run the front end against the DEV db. Use parts 2 / 3 for that after having run the notebook once from start to end.

In [1]:
import configparser
import datetime as dt
import sys
import collections
from pathlib import Path
from typing import Union, Optional, Iterable, List, Tuple

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from sqlalchemy import func, distinct

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [3]:
sys.path.append(str(Path.cwd().parent))

from src.db_declaration import (
    Base, Artist, CreditTrx, Genre, Label, Record, RecordFormat, 
    ArtistRecordLink, ArtistGenreLink, GenreLabelLink, LabelRecordLink
)
from src import db_functions
from src import db_connect
from src.apps import crud_app

CONFIG_PATH = Path.cwd().parent / "config.yaml"

In [4]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## (Re-)Create A Test DB

### Connect And Reset

Because we connect from `dev`subfolder, I work with the sqlalchemy `create_engine` function directly.

In [5]:
# session.close()

db_params = {"REL_PATH": "DeafDiscoBase.db"}
engine = db_connect.create_engine(db_params)
session = db_connect.create_session(engine)

# Check
engine
session

Engine(sqlite:///C:\Users\r2d4\OneDrive\code\projects\20-02_disco\dev\DeafDiscoBase.db)

In [6]:
db_functions._drop_and_reset_DB(engine, Base)

## Insert And Remove

### Load and Insert 3 Test Records, assert

In [7]:
record_data = pd.read_parquet(
    r"C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\back-up\record_data_2021-02-17-09-02-53.parquet"
)
test_records = record_data[record_data.index.isin([1, 2, 225, 301])].copy()
test_records.head()

,artist,artist_country,title,genre,label,year,record_format,vinyl_color,lim_edition,number,remarks,purchase_date,price,rating,is_digitized,is_active
record_id,,,,,,,,,,,,,,,,
1,[Dismember],[NA],Pieces,Death Metal,[NA],1992,"12""",None,None,None,None,1992-01-01,35.00,nan,False,True
2,[Dismember],[NA],Skin Her Alive,Death Metal,[NA],1991,"7"" Pic",None,None,None,None,1992-01-01,15.00,nan,False,True
225,[Petrification],[NA],Hollow of The Void,Death Metal,[Dawnbreed / Sentient Ruin],2018,LP,green transparent with black haze,200,None,None,2019-07-01,25.00,10.00,True,True
301,"[Coffins, Depression]","[NA, Germany]",Split,Death Metal,[Hells Headbangers],2021,LP,white with black splatter,100,None,None,2021-01-20,20.00,nan,True,True


In [8]:
db_functions._insert_record_data_with_sqlalchemy_orm(session, test_records)

C:\Users\r2d4\miniconda3\envs\py3\lib\site-packages\sqlalchemy\sql\sqltypes.py:660: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


In [9]:
assert session.query(Record).count() == 4
assert session.query(ArtistRecordLink).count() == 5

### Insertion of 2 Credit Addition Trx, assert

In [10]:
# to_delete = session.query(CreditTrx).filter(CreditTrx.credit_trx_id == 8).one()
# session.delete(to_delete)
# session.commit()

In [11]:
# Initial trx, 11 days ago

addition_trx = CreditTrx(
    credit_trx_date=dt.datetime.today().date() - dt.timedelta(11),
    credit_trx_type="Addition",
    credit_value=1,
    credit_saldo=1,
    record_id=np.nan
)
session.add(addition_trx)

# And a regular interval addition
db_functions.add_regular_credits(session)

session.commit()

Creating 'Addition' Trx for: 2021-03-08


In [12]:
pd.read_sql("credit_trx", engine)

,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
0,1,1992-01-01,Initial Load,0.00,0.00,1.00,2021-03-09 14:04:11,NaT
1,2,1992-01-01,Initial Load,0.00,0.00,2.00,2021-03-09 14:04:11,NaT
2,3,2019-07-01,Initial Load,0.00,0.00,3.00,2021-03-09 14:04:11,NaT
3,4,2021-01-20,Initial Load,0.00,0.00,4.00,2021-03-09 14:04:11,NaT
4,5,2021-02-26,Addition,1.00,1.00,nan,2021-03-09 14:04:12,NaT
5,6,2021-03-08,Addition,1.00,2.00,nan,2021-03-09 14:04:12,NaT


In [13]:
assert session.query(CreditTrx).count()
assert session.query(func.sum(CreditTrx.credit_value)).all()[0][0] == 2
assert session.query(func.max(CreditTrx.credit_saldo)).all()[0][0] == 2

### [Display of Tables]

In [14]:
pd.read_sql("records", engine)

,record_id,title,year,genre_id,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,rating,is_digitized,is_active,created_at,updated_at
0,1,Pieces,1992,1,1,None,None,None,None,1992-01-01,35,nan,0,1,2021-03-09 14:04:11,NaT
1,2,Skin Her Alive,1991,1,2,None,None,None,None,1992-01-01,15,nan,0,1,2021-03-09 14:04:11,NaT
2,3,Hollow of The Void,2018,1,3,green transparent with black haze,200,None,None,2019-07-01,25,10.00,1,1,2021-03-09 14:04:11,NaT
3,4,Split,2021,1,3,white with black splatter,100,None,None,2021-01-20,20,nan,1,1,2021-03-09 14:04:11,NaT


In [15]:
pd.read_sql("artists", engine)

,artist_id,artist_name,artist_country,created_at,updated_at
0,1,Dismember,NA,2021-03-09 14:04:11,NaT
1,2,Petrification,NA,2021-03-09 14:04:11,NaT
2,3,Coffins,NA,2021-03-09 14:04:11,NaT
3,4,Depression,Germany,2021-03-09 14:04:11,NaT


In [16]:
pd.read_sql("labels", engine)

,label_id,label_name,created_at,updated_at
0,1,NA,2021-03-09 14:04:11,NaT
1,2,Dawnbreed / Sentient Ruin,2021-03-09 14:04:11,NaT
2,3,Hells Headbangers,2021-03-09 14:04:11,NaT


In [17]:
pd.read_sql("artist_label_link", engine)

,artist_id,label_id
0,1,1
1,2,2
2,3,3
3,4,3


In [18]:
pd.read_sql("artist_label_link", engine)

,artist_id,label_id
0,1,1
1,2,2
2,3,3
3,4,3


In [19]:
session.close()

### Removal of Existing Record, assert

Necessary cols: trx_type, credit_value, title, artist, date

In [20]:
test_removal = {
    "trx_type": "Remove",
    "credit_value": 1,
    "artist": "Coffins",
    "title": "Split",
#     "year": 1993,
    "removal_date": dt.datetime.today().date()
}

In [21]:
db_functions.set_record_to_inactive(session, test_removal)
session.query(func.count(distinct(Record.is_active))).all()[0][0] == 3
assert len(session.query(CreditTrx).all()) == 7

Record set to inactive.


False

In [22]:
# session.query(CreditTrx).all()[-3:]

### Reactivation of inactive Record [OPEN]

In [23]:
# ATTENTION It has to be possible ro re-add inactive records! (and to pay for it in credits!)

### [Display Of Tables]

In [24]:
pd.read_sql("records", engine)

,record_id,title,year,genre_id,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,rating,is_digitized,is_active,created_at,updated_at
0,1,Pieces,1992,1,1,None,None,None,None,1992-01-01,35,nan,0,1,2021-03-09 14:04:11,NaT
1,2,Skin Her Alive,1991,1,2,None,None,None,None,1992-01-01,15,nan,0,1,2021-03-09 14:04:11,NaT
2,3,Hollow of The Void,2018,1,3,green transparent with black haze,200,None,None,2019-07-01,25,10.00,1,1,2021-03-09 14:04:11,NaT
3,4,Split,2021,1,3,white with black splatter,100,None,None,2021-01-20,20,nan,1,0,2021-03-09 14:04:11,2021-03-09 14:04:14


In [25]:
pd.read_sql("artists", engine)

,artist_id,artist_name,artist_country,created_at,updated_at
0,1,Dismember,NA,2021-03-09 14:04:11,NaT
1,2,Petrification,NA,2021-03-09 14:04:11,NaT
2,3,Coffins,NA,2021-03-09 14:04:11,NaT
3,4,Depression,Germany,2021-03-09 14:04:11,NaT


In [26]:
pd.read_sql("artist_record_link", engine)

,artist_id,record_id
0,1,1
1,1,2
2,2,3
3,3,4
4,4,4


In [27]:
pd.read_sql("genres", engine)

,genre_id,genre_name,created_at,updated_at
0,1,Death Metal,2021-03-09 14:04:11,NaT


In [28]:
pd.read_sql("artist_genre_link", engine)

,artist_id,genre_id
0,1,1
1,2,1
2,3,1
3,4,1


In [29]:
pd.read_sql("formats", engine)

,format_id,format_name,created_at,updated_at
0,1,"12""",2021-03-09 14:04:11,NaT
1,2,"7"" Pic",2021-03-09 14:04:11,NaT
2,3,LP,2021-03-09 14:04:11,NaT


In [30]:
pd.read_sql("labels", engine)

,label_id,label_name,created_at,updated_at
0,1,NA,2021-03-09 14:04:11,NaT
1,2,Dawnbreed / Sentient Ruin,2021-03-09 14:04:11,NaT
2,3,Hells Headbangers,2021-03-09 14:04:11,NaT


In [31]:
pd.read_sql("label_record_link", engine)

,label_id,record_id
0,1,1
1,1,2
2,2,3
3,3,4


In [32]:
pd.read_sql("artist_label_link", engine)

,artist_id,label_id
0,1,1
1,2,2
2,3,3
3,4,3


### Update Of Record, assert

In [33]:
artist = "Dismember"
title ="Pieces"
trx_type = "Update"

record = db_functions.fetch_a_record_from_the_shelf(session, artist, title)
assert record.record_id == 1

artist = "; ".join([artist.artist_name for artist in record.artists])
artist_country = "; ".join([artist.artist_country for artist in record.artists])
title = record.title    
genre = value=record.genre.genre_name
label ="; ".join([label.label_name for label in record.labels])
year = record.year
record_format =record.record_format.format_name
vinyl_color = record.vinyl_color
lim_edition = record.lim_edition
number = record.number
remarks = record.remarks
price = float(record.price)
is_digitized = record.is_digitized
rating = record.rating
is_active = record.is_active
purchase_date = record.purchase_date

record_data = {
    "trx_type": trx_type,
    "artist": artist if artist != "" else "NA",
    "artist_country": artist_country if artist_country != "" else "NA",
    "title": title if title != "" else None,
    "genre": genre,
    "label": label if label != "" else "NA",
    "year": year,
    "record_format": record_format,
    "vinyl_color": vinyl_color if vinyl_color != "" else None,
    "lim_edition": lim_edition if lim_edition != "" else None,
    "number": number if number != "" else None,
    "remarks": remarks if remarks != "" else None,
    "purchase_date": purchase_date,
    "price": price,
    "rating": rating if rating != "" else None,
    "is_digitized": is_digitized,
    "is_active": is_active,
    # "credit_value": credit_value,  # we dont return a cretid value yet
}

record_data

{'trx_type': 'Update',
 'artist': 'Dismember',
 'artist_country': 'NA',
 'title': 'Pieces',
 'genre': 'Death Metal',
 'label': 'NA',
 'year': 1992,
 'record_format': '12"',
 'vinyl_color': None,
 'lim_edition': None,
 'number': None,
 'remarks': None,
 'purchase_date': datetime.date(1992, 1, 1),
 'price': 35.0,
 'rating': None,
 'is_digitized': 0,
 'is_active': 1}

In [34]:
record_data["artist_country"] = "Sweden"
record_data["label"] = "Nuclear Blast; Bogus"

In [35]:
record_data = crud_app.prepare_record_data_for_DB_insert(record_data)
record_data

{'trx_type': 'Update',
 'artist': ['Dismember'],
 'artist_country': ['Sweden'],
 'title': 'Pieces',
 'genre': 'Death Metal',
 'label': ['Nuclear Blast', 'Bogus'],
 'year': 1992,
 'record_format': '12"',
 'vinyl_color': None,
 'lim_edition': None,
 'number': None,
 'remarks': None,
 'purchase_date': datetime.date(1992, 1, 1),
 'price': 35.0,
 'rating': None,
 'is_digitized': 0,
 'is_active': 1}

In [36]:
db_functions.update_record(session, record_data)

In [37]:
result = session.query(Record).filter(Record.record_id == 1).one_or_none()

assert session.query(Label.label_name).filter(Label.label_id == 5).one_or_none()[0] == "Bogus"
assert [artist.artist_name for artist in result.artists] == ["Dismember"]
assert [artist.artist_country for artist in result.artists] == ["Sweden"]

## (Effective) DELETION of Record, assert

In [38]:
to_delete = session.query(Record).filter(Record.record_id == 4).one_or_none()
session.delete(to_delete)
session.commit()

assert session.query(Record).count() == 3

In [39]:
pd.read_sql("labels", engine)
pd.read_sql("artists", engine)
pd.read_sql("artist_label_link", engine)
pd.read_sql("credit_trx", engine)

assert session.query(Record).count() == 3
assert session.query(ArtistRecordLink).count() == 3
assert session.query(CreditTrx).count() == 5

,label_id,label_name,created_at,updated_at
0,1,NA,2021-03-09 14:04:11,NaT
1,2,Dawnbreed / Sentient Ruin,2021-03-09 14:04:11,NaT
2,3,Hells Headbangers,2021-03-09 14:04:11,NaT
3,4,Nuclear Blast,2021-03-09 14:04:17,NaT
4,5,Bogus,2021-03-09 14:04:17,NaT


,artist_id,artist_name,artist_country,created_at,updated_at
0,1,Dismember,Sweden,2021-03-09 14:04:11,2021-03-09 14:04:17
1,2,Petrification,NA,2021-03-09 14:04:11,NaT
2,3,Coffins,NA,2021-03-09 14:04:11,NaT
3,4,Depression,Germany,2021-03-09 14:04:11,NaT


,artist_id,label_id
0,1,1
1,2,2
2,3,3
3,4,3
4,1,4
5,1,5


,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
0,1,1992-01-01,Initial Load,0.00,0.00,1.00,2021-03-09 14:04:11,NaT
1,2,1992-01-01,Initial Load,0.00,0.00,2.00,2021-03-09 14:04:11,NaT
2,3,2019-07-01,Initial Load,0.00,0.00,3.00,2021-03-09 14:04:11,NaT
3,5,2021-02-26,Addition,1.00,1.00,nan,2021-03-09 14:04:12,NaT
4,6,2021-03-08,Addition,1.00,2.00,nan,2021-03-09 14:04:12,NaT


In [40]:
pd.read_sql("genres", engine)

,genre_id,genre_name,created_at,updated_at
0,1,Death Metal,2021-03-09 14:04:11,NaT


In [41]:
to_delete = session.query(Record).filter(Record.record_id == 1).one_or_none()
session.delete(to_delete)
session.commit()

assert session.query(Record).count() == 2

In [42]:
pd.read_sql("records", engine)
pd.read_sql("labels", engine)
pd.read_sql("artists", engine)
pd.read_sql("artist_label_link", engine)
pd.read_sql("credit_trx", engine)
pd.read_sql("formats", engine)

,record_id,title,year,genre_id,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,rating,is_digitized,is_active,created_at,updated_at
0,2,Skin Her Alive,1991,1,2,None,None,None,None,1992-01-01,15,nan,0,1,2021-03-09 14:04:11,NaT
1,3,Hollow of The Void,2018,1,3,green transparent with black haze,200,None,None,2019-07-01,25,10.00,1,1,2021-03-09 14:04:11,NaT


,label_id,label_name,created_at,updated_at
0,1,NA,2021-03-09 14:04:11,NaT
1,2,Dawnbreed / Sentient Ruin,2021-03-09 14:04:11,NaT
2,3,Hells Headbangers,2021-03-09 14:04:11,NaT
3,4,Nuclear Blast,2021-03-09 14:04:17,NaT
4,5,Bogus,2021-03-09 14:04:17,NaT


,artist_id,artist_name,artist_country,created_at,updated_at
0,1,Dismember,Sweden,2021-03-09 14:04:11,2021-03-09 14:04:17
1,2,Petrification,NA,2021-03-09 14:04:11,NaT
2,3,Coffins,NA,2021-03-09 14:04:11,NaT
3,4,Depression,Germany,2021-03-09 14:04:11,NaT


,artist_id,label_id
0,1,1
1,2,2
2,3,3
3,4,3
4,1,4
5,1,5


,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
0,2,1992-01-01,Initial Load,0.00,0.00,2.00,2021-03-09 14:04:11,NaT
1,3,2019-07-01,Initial Load,0.00,0.00,3.00,2021-03-09 14:04:11,NaT
2,5,2021-02-26,Addition,1.00,1.00,nan,2021-03-09 14:04:12,NaT
3,6,2021-03-08,Addition,1.00,2.00,nan,2021-03-09 14:04:12,NaT


,format_id,format_name,created_at,updated_at
0,1,"12""",2021-03-09 14:04:11,NaT
1,2,"7"" Pic",2021-03-09 14:04:11,NaT
2,3,LP,2021-03-09 14:04:11,NaT


In [43]:
result = session.query(Label)
for r in result:
    print(r.artists)
    
print("")
    
result = session.query(Genre)
for r in result:
    print(r.artists)

[<Artist(artist_id=1, artist_name=Dismember)>]
[<Artist(artist_id=2, artist_name=Petrification)>]
[<Artist(artist_id=3, artist_name=Coffins)>, <Artist(artist_id=4, artist_name=Depression)>]
[<Artist(artist_id=1, artist_name=Dismember)>]
[<Artist(artist_id=1, artist_name=Dismember)>]

[<Artist(artist_id=1, artist_name=Dismember)>, <Artist(artist_id=2, artist_name=Petrification)>, <Artist(artist_id=3, artist_name=Coffins)>, <Artist(artist_id=4, artist_name=Depression)>]


**ATTENTION:** Altough there are NO records for artists Coffins and Depression left, these artists are still listed as such in the DB, as is their relation with the Label Hells Headbangers (that also has no record left here.) That is not ideal.

For the credit_trx (and the ratings, if they were implemented yet) the behaviour is as desired.

---

In [44]:
brk

NameError: name 'brk' is not defined

In [ ]:
result = session.query(Record).filter(Record.record_id == 2).one()

# HERE I MANUALLY OVERWRITE AN ARTISTS RELATIONSHIP, WHILE ALSO CREATING A NEW ARTIST ON THE FLY
result.artists = [Artist(artist_id=5, artist_name="Fake-Artist")]
print(result)

In [ ]:
type(result.artists)

In [ ]:
result = session.query(Artist).all()
for r in result:
    print(r, r.artist_country)

In [ ]:
type(result[0].artist_country)

In [ ]:
session.close()